In [2]:
from atproto import Client
import os
from dotenv import load_dotenv
import requests
import json


client = Client()
client.login("bluerelief.bsky.social", "HardCoders1!")

ProfileViewDetailed(did='did:plc:kxi2747kvqtdehywvhdikgis', handle='bluerelief.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:kxi2747kvqtdehywvhdikgis/bafkreie6bajieexpa5zkus2lowc36zmlkravj4rairevkmvybugyxvl2hi@jpeg', banner=None, created_at='2025-09-21T20:33:12.699Z', description=None, display_name='', followers_count=1, follows_count=1, indexed_at='2025-09-21T20:33:12.699Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=0, status=None, verification=None, viewer=ViewerState(activity_subscription=None, blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=N

In [3]:
import requests

username = os.getenv("BlueSky_Username")
app_password = os.getenv("BlueSky_Password")

response = requests.post(
    "https://bsky.social/xrpc/com.atproto.server.createSession",
    json={"identifier": username, "password": app_password}
)
response.raise_for_status()
session_data = response.json()
access_token = session_data["accessJwt"]

headers = {"Authorization": f"Bearer {access_token}"}


In [4]:
hashtag = "#earthquake"  # include the `#`
params = {
    "q": hashtag,
    "limit": 50
}

response = requests.get(
    "https://bsky.social/xrpc/app.bsky.feed.searchPosts",
    headers=headers,
    params=params
)
response.raise_for_status()
posts = response.json().get("posts", [])

# Extract post text
for idx, post in enumerate(posts, 1):
    text = post["record"]["text"]
    print(f"{idx}. {text}\n")

1. #Earthquake M4.6 | 16 km E of San Vicente, Mexico | 4m ago | Oct 6, 2025 04:53 pm (Universal Time)
Did you feel it? Submit a report!
#moderate #quake #M2+ #M3+ #M4+ #Mexico #Estado_de_Baja_California
Quake feed for Mexico: @mexico.earthquakes.live

2. Tsunami Info Stmt- 105 miles SW of El Centro, California area
M4.6 earthquake 105 miles SW of El Centro, California 09:53PDT Oct 6 Depth 1km: Tsunami NOT expected

3. 2 minutes 35 seconds ago -  Baja California, Mexico  MLv 4.5  31.33N 116.28W  0km 2025/10/06  16:53:28 - For updates, see http://www.seisme.nc/index.php?option=com_alertes&view=alertes&Itemid=184&ids=ird2025trpmbo&lang=en #séisme #earthquake #sismo #지진 #地震

4. M4.1 #earthquake detected 26 km ESE of Benito García (El Zorrillo), Mexico at 10/6/2025 9:53:32 AM PDT

This reflects data from the U.S. Geological Survey that is subject to change. Visit the USGS page linked below for the latest #quake information

5. #Earthquake M4.1 | 26 km ESE of Benito García (El Zorrillo), Mex

In [5]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load variables from .env
load_dotenv()

# Prepare the prompt for Gemini
prompt = (
    "Extract the location of the disaster, time, and rate the severity on a scale of 1-5:\n\n"
    + "\n".join([f"{idx}. {post['record']['text']}" for idx, post in enumerate(posts, 1)])
)

# Initialize Gemini model (make sure you have set your API key)
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content(prompt)
print(response.text)

c:\Users\kumar\OneDrive\Documents\GitHub\BlueSky-Integration\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here's the extracted information and severity ratings for each distinct disaster event:

**Severity Scale:**
*   **1:** Minor (Magnitude < 3.0)
*   **2:** Light (Magnitude 3.0-3.9)
*   **3:** Moderate (Magnitude 4.0-4.9)
*   **4:** Strong (Magnitude 5.0-5.9)
*   **5:** Severe (Magnitude 6.0+)

---

1.  **Location:** 16 km E of San Vicente, Mexico (also 105 miles SW of El Centro, California area / Baja California, Mexico at 31.33N 116.28W)
    **Time:** Oct 6, 2025 04:53 pm (Universal Time)
    **Severity:** 3 (M4.6)

2.  **Location:** 26 km ESE of Benito García (El Zorrillo), Mexico
    **Time:** Oct 6, 2025 04:53 pm (Universal Time)
    **Severity:** 3 (M4.1)

3.  **Location:** Far E Off North Honshu, Japan
    **Time:** Oct 7, 2025 01:44 am (GMT +9)
    **Severity:** 3 (M4.1)

4.  **Location:** Off Coast of Jalisco, Mexico
    **Time:** Oct 6, 2025 04:28 pm (Universal Time)
    **Severity:** 3 (M4.0)

5.  **Location:** 32 km al O de Ollagüe, Chile (-21.25, -68.56)
    **Time:** Oct 6

In [19]:
url = "https://huggingface.co/api/models/mistralai/Mistral-7B-Instruct-v0.2"
r = requests.get(url)
print(r.status_code)
print(r.json().get("id"))

200
mistralai/Mistral-7B-Instruct-v0.2


In [18]:
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
r = requests.post(url, headers={"Authorization": f"Bearer {os.getenv("HUGGING_FACE_TOKEN")}"}, json={"inputs": "Hello"})
print(r.status_code, r.text)

404 Not Found


In [24]:
from mistralai import Mistral
from dotenv import load_dotenv
import os
import json
from pprint import pprint

load_dotenv()

# ✅ Get your API key
api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise RuntimeError("Missing MISTRAL_API_KEY in .env")

# ✅ Initialize client
client = Mistral(api_key=api_key)

# ✅ Make sure 'posts' exists
if 'posts' not in globals():
    raise RuntimeError("`posts` not found. Run the cell that fetches posts first.")

# Build the prompt (robust to posts being dicts or plain strings)
examples = []
for idx, post in enumerate(posts, 1):
    text = ""
    # post can be a dict (from API) or a string (preprocessed) or something else
    if isinstance(post, dict):
        # common BlueSky shape: post['record']['text']
        record = post.get('record') or post.get('value') or {}
        if isinstance(record, dict):
            # try common keys
            if 'text' in record and isinstance(record.get('text'), str):
                text = record.get('text') or ''
            elif 'content' in record and isinstance(record.get('content'), str):
                text = record.get('content') or ''
            else:
                # fallback: stringify the record cleanly
                try:
                    text = json.dumps(record, ensure_ascii=False)
                except Exception:
                    text = str(record)
        else:
            text = str(record)
    elif isinstance(post, str):
        text = post
    else:
        try:
            text = json.dumps(post, ensure_ascii=False)
        except Exception:
            text = str(post)

    examples.append(f"{idx}. {text}")

prompt = (
    "You are an assistant that extracts structured information from short social media posts.\n"
    "For each post, extract the following fields: 'index' (int), 'text' (string), 'location' (string or null), "
    "'time' (string or null), 'severity' (integer 1-5 or null), and 'notes' (optional string).\n"
    "Return ONLY valid JSON: an array of objects with those keys. Do not include explanations.\n\n"
    "Posts:\n" + "\n".join(examples)
)

# ✅ Call Mistral Instruct model (7B)
response = client.chat.complete(
    model="mistral-small",  # 7B Instruct (hosted by Mistral)
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0,
    max_tokens=800,
)

# ✅ Extract text
output_text = response.choices[0].message.content

# ✅ Try parsing JSON
try:
    parsed = json.loads(output_text)
    pprint(parsed)
except json.JSONDecodeError:
    print("⚠️ Could not parse JSON. Raw output:")
    print(output_text)


[{'index': 1,
  'location': '16 km E of San Vicente, Mexico',
  'notes': None,
  'severity': 4,
  'text': '#Earthquake M4.6',
  'time': '4m ago | Oct 6, 2025 04:53 pm (Universal Time)'},
 {'index': 2,
  'location': '105 miles SW of El Centro, California area M4.6 earthquake',
  'notes': 'Tsunami NOT expected',
  'severity': None,
  'text': 'Tsunami Info Stmt',
  'time': '09:53PDT Oct 6'},
 {'index': 3,
  'location': '31.33N 116.28W 0km',
  'notes': None,
  'severity': 4,
  'text': 'Baja California, Mexico MLv 4.5',
  'time': '2 minutes 35 seconds ago | 2025/10/06 16:53:28'},
 {'index': 4,
  'location': '26 km ESE of Benito García (El Zorrillo), Mexico',
  'notes': None,
  'severity': 4,
  'text': 'M4.1 #earthquake detected',
  'time': '10/6/2025 9:53:32 AM PDT'}]


In [29]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

# Load env
load_dotenv()

# Accept either OPENROUTER_API_KEY or LLaMa_API_KEY for compatibility
api_key = os.getenv("OPENROUTER_API_KEY") or os.getenv("LLaMa_API_KEY") or os.getenv("OPENROUTER_KEY")
if not api_key:
    raise RuntimeError("Missing OpenRouter API key. Set OPENROUTER_API_KEY or LLaMa_API_KEY in .env")

base_url = os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")
model = os.getenv("OPENROUTER_MODEL", "meta-llama/llama-3.3-70b-instruct:free")
referer = os.getenv("OPENROUTER_REFERER")
title = os.getenv("OPENROUTER_TITLE")

# Initialize client
client = OpenAI(base_url=base_url, api_key=api_key)

# Ensure posts exist
if 'posts' not in globals():
    raise RuntimeError("`posts` not found. Run the Bluesky fetch cell before this one.")

# Build examples from posts (robust to different shapes)
examples = []
for idx, post in enumerate(posts, 1):
    text = ""
    if isinstance(post, dict):
        record = post.get('record') or post.get('value') or {}
        if isinstance(record, dict):
            if isinstance(record.get('text'), str):
                text = record.get('text')
            elif isinstance(record.get('content'), str):
                text = record.get('content')
            else:
                try:
                    text = json.dumps(record, ensure_ascii=False)
                except Exception:
                    text = str(record)
        else:
            text = str(record)
    elif isinstance(post, str):
        text = post
    else:
        try:
            text = json.dumps(post, ensure_ascii=False)
        except Exception:
            text = str(post)
    examples.append(f"{idx}. {text}")

# Build prompt asking for strict JSON output
prompt = (
    "You are an assistant that extracts structured information from short social media posts.\n"
    "For each post provided, extract: 'index' (int), 'text' (string), 'location' (string or null), 'time' (string or null), 'severity' (integer 1-5 or null), and 'notes' (optional string).\n"
    "Return ONLY valid JSON: an array of objects with those keys. No extra commentary.\n\n"
    "Posts:\n"
    + "\n".join(examples)
)

messages = [{"role": "user", "content": prompt}]

# Optional extra headers
extra_headers = {}
if referer:
    extra_headers["HTTP-Referer"] = referer
if title:
    extra_headers["X-Title"] = title

print(f"Sending {len(messages)} message(s) to {model} at {base_url}")

try:
    completion = client.chat.completions.create(
        extra_headers=extra_headers,
        extra_body={},
        model=model,
        messages=messages,
    )
except Exception as e:
    raise RuntimeError(f"OpenRouter request failed: {e}")

# Safely extract the content
content = None
try:
    content = completion.choices[0].message.content
except Exception:
    # fallback: try to stringify the object
    try:
        content = json.dumps(completion, default=str)
    except Exception:
        content = str(completion)

# Try to parse JSON result, otherwise print raw
try:
    parsed = json.loads(content)
    from pprint import pprint
    pprint(parsed)
except Exception:
    print("--- Raw model output ---")
    print(content)
    print("------------------------")


Sending 1 message(s) to meta-llama/llama-3.3-70b-instruct:free at https://openrouter.ai/api/v1
[{'index': 1,
  'location': '16 km E of San Vicente, Mexico',
  'notes': None,
  'severity': 4,
  'text': '#Earthquake M4.6 | 16 km E of San Vicente, Mexico | 4m ago | Oct 6, '
          '2025 04:53 pm (Universal Time)',
  'time': '4m ago | Oct 6, 2025 04:53 pm'},
 {'index': 2,
  'location': '105 miles SW of El Centro, California',
  'notes': 'Tsunami NOT expected',
  'severity': 4,
  'text': 'Tsunami Info Stmt- 105 miles SW of El Centro, California area M4.6 '
          'earthquake 09:53PDT Oct 6 Depth 1km: Tsunami NOT expected',
  'time': '09:53PDT Oct 6'},
 {'index': 3,
  'location': 'Baja California, Mexico',
  'notes': None,
  'severity': 4,
  'text': '2 minutes 35 seconds ago -  Baja California, Mexico  MLv 4.5  '
          '31.33N 116.28W  0km 2025/10/06  16:53:28',
  'time': '2 minutes 35 seconds ago | 2025/10/06  16:53:28'},
 {'index': 4,
  'location': '26 km ESE of Benito García (El

In [32]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from pprint import pprint

# Load env
load_dotenv()

# Get API key
api_key = os.getenv("OPENROUTER_API_KEY") or os.getenv("LLaMa_API_KEY")
if not api_key:
    raise RuntimeError("⚠️ OPENROUTER_API_KEY (or LLaMa_API_KEY) not found in environment!")

# Create client
client = OpenAI(base_url=os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1"), api_key=api_key)

# Ensure posts are available
if 'posts' not in globals():
    raise RuntimeError("`posts` not found. Run the Bluesky fetch cell before this one.")

# Build examples from posts (robust)
examples = []
for idx, post in enumerate(posts, 1):
    text = ""
    if isinstance(post, dict):
        record = post.get('record') or post.get('value') or {}
        if isinstance(record, dict):
            if isinstance(record.get('text'), str):
                text = record.get('text')
            elif isinstance(record.get('content'), str):
                text = record.get('content')
            else:
                try:
                    text = json.dumps(record, ensure_ascii=False)
                except Exception:
                    text = str(record)
        else:
            text = str(record)
    elif isinstance(post, str):
        text = post
    else:
        try:
            text = json.dumps(post, ensure_ascii=False)
        except Exception:
            text = str(post)
    examples.append(f"{idx}. {text}")

# Build instruction prompt
prompt = (
    "You are an assistant that extracts structured information from short social media posts.\n"
    "For each post provided, extract the following fields: 'index' (int), 'text' (string), 'location' (string or null), 'time' (string or null), 'severity' (integer 1-5 or null), and 'notes' (optional string).\n"
    "Return ONLY valid JSON: an array of objects with those keys. Do not include any extra explanation.\n\n"
    "Posts:\n" + "\n".join(examples)
)

# Prepare request
model = os.getenv('OPENROUTER_MODEL', 'openai/gpt-oss-20b:free')
extra_headers = {
    "HTTP-Referer": os.getenv('OPENROUTER_REFERER', ''),
    "X-Title": os.getenv('OPENROUTER_TITLE', ''),
}
# remove empty headers
extra_headers = {k: v for k, v in extra_headers.items() if v}

messages = [{"role": "user", "content": prompt}]
print(f"Sending {len(messages)} message(s) to {model}")

# Call OpenRouter
try:
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        extra_headers=extra_headers,
    )
except Exception as e:
    raise RuntimeError(f"OpenRouter request failed: {e}")

# Extract content
content = None
try:
    content = completion.choices[0].message.content
except Exception:
    try:
        content = json.dumps(completion, default=str)
    except Exception:
        content = str(completion)

# Try to parse JSON
try:
    parsed = json.loads(content)
    pprint(parsed)
except Exception:
    print('--- Raw model output ---')
    print(content)
    print('------------------------')


Sending 1 message(s) to openai/gpt-oss-20b:free
[{'index': 1,
  'location': '16 km E of San Vicente, Mexico',
  'notes': None,
  'severity': 4,
  'text': '#Earthquake M4.6 | 16 km E of San Vicente, Mexico | 4m ago | Oct 6, '
          '2025 04:53 pm (Universal Time)',
  'time': 'Oct 6, 2025 04:53 pm (Universal Time)'},
 {'index': 2,
  'location': '105 miles SW of El Centro, California area',
  'notes': 'Tsunami NOT expected',
  'severity': 4,
  'text': 'Tsunami Info Stmt- 105 miles SW of El Centro, California area M4.6 '
          'earthquake 09:53PDT Oct 6 Depth 1km: Tsunami NOT expected',
  'time': '09:53 PDT Oct 6'},
 {'index': 3,
  'location': 'Baja California, Mexico',
  'notes': None,
  'severity': 4,
  'text': '2 minutes 35 seconds ago - Baja California, Mexico MLv 4.5 31.33N '
          '116.28W 0km 2025/10/06 16:53:28',
  'time': '2025/10/06 16:53:28'},
 {'index': 4,
  'location': '26 km ESE of Benito García (El Zorrillo), Mexico',
  'notes': None,
  'severity': 4,
  'text': '

In [33]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from pprint import pprint

# Load env
load_dotenv()

# Get API key
api_key = os.getenv("OPENROUTER_API_KEY") or os.getenv("LLaMa_API_KEY")
if not api_key:
    raise RuntimeError("⚠️ OPENROUTER_API_KEY (or LLaMa_API_KEY) not found in environment!")

# Create client
client = OpenAI(base_url=os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1"), api_key=api_key)

# Ensure posts are available
if 'posts' not in globals():
    raise RuntimeError("`posts` not found. Run the Bluesky fetch cell before this one.")

# Build examples from posts (robust)
examples = []
for idx, post in enumerate(posts, 1):
    text = ""
    if isinstance(post, dict):
        record = post.get('record') or post.get('value') or {}
        if isinstance(record, dict):
            if isinstance(record.get('text'), str):
                text = record.get('text')
            elif isinstance(record.get('content'), str):
                text = record.get('content')
            else:
                try:
                    text = json.dumps(record, ensure_ascii=False)
                except Exception:
                    text = str(record)
        else:
            text = str(record)
    elif isinstance(post, str):
        text = post
    else:
        try:
            text = json.dumps(post, ensure_ascii=False)
        except Exception:
            text = str(post)
    examples.append(f"{idx}. {text}")

# Build instruction prompt
prompt = (
    "You are an assistant that extracts structured information from short social media posts.\n"
    "For each post provided, extract the following fields: 'index' (int), 'text' (string), 'location' (string or null), 'time' (string or null), 'severity' (integer 1-5 or null), and 'notes' (optional string).\n"
    "Return ONLY valid JSON: an array of objects with those keys. Do not include any extra explanation.\n\n"
    "Posts:\n" + "\n".join(examples)
)

# Prepare request
model = os.getenv('OPENROUTER_MODEL', 'microsoft/mai-ds-r1:free')
extra_headers = {
    "HTTP-Referer": os.getenv('OPENROUTER_REFERER', ''),
    "X-Title": os.getenv('OPENROUTER_TITLE', ''),
}
# remove empty headers
extra_headers = {k: v for k, v in extra_headers.items() if v}

messages = [{"role": "user", "content": prompt}]
print(f"Sending {len(messages)} message(s) to {model}")

# Call OpenRouter
try:
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        extra_headers=extra_headers,
    )
except Exception as e:
    raise RuntimeError(f"OpenRouter request failed: {e}")

# Extract content
content = None
try:
    content = completion.choices[0].message.content
except Exception:
    try:
        content = json.dumps(completion, default=str)
    except Exception:
        content = str(completion)

# Try to parse JSON
try:
    parsed = json.loads(content)
    pprint(parsed)
except Exception:
    print('--- Raw model output ---')
    print(content)
    print('------------------------')


Sending 1 message(s) to microsoft/mai-ds-r1:free
[{'index': 1,
  'location': '16 km E of San Vicente, Mexico',
  'notes': None,
  'severity': 4,
  'text': '#Earthquake M4.6 | 16 km E of San Vicente, Mexico | 4m ago | Oct 6, '
          '2025 04:53 pm (Universal Time)',
  'time': 'Oct 6, 2025 04:53 pm (Universal Time)'},
 {'index': 2,
  'location': '105 miles SW of El Centro, California',
  'notes': 'Tsunami NOT expected; Depth 1km',
  'severity': 4,
  'text': 'Tsunami Info Stmt- 105 miles SW of El Centro, California area M4.6 '
          'earthquake 09:53PDT Oct 6 Depth 1km: Tsunami NOT expected',
  'time': '09:53PDT Oct 6'},
 {'index': 3,
  'location': 'Baja California, Mexico',
  'notes': 'Coordinates: 31.33N 116.28W; Depth: 0km',
  'severity': 4,
  'text': '2 minutes 35 seconds ago -  Baja California, Mexico  MLv 4.5  '
          '31.33N 116.28W  0km 2025/10/06  16:53:28',
  'time': '2025/10/06 16:53:28'},
 {'index': 4,
  'location': '26 km ESE of Benito García (El Zorrillo), Mexico